In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1 import keras

config = ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement=True

keras.backend.set_session(InteractiveSession(config=config))
session = InteractiveSession(config=config)

C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\tensorflow\python\f

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator #Augmentation
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)
                     #y_train為(50000,1)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train)
        std = np.std(X_train)
        X_train = (X_train-mean)/(std+1e-7)  #1e-7為1*10的-7次方,這是為了防止當std=0時會除以0
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()    # y_train.shape變為(50000,10)
y_test = one_hot.transform(y_test).toarray()


classifier = Sequential()

#卷積組合
classifier.add(Conv2D(32, (3,3), input_shape=(32,32,3), activation='relu'))
classifier.add(BatchNormalization())   # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Conv2D(32, (3,3), activation='relu'))
classifier.add(BatchNormalization())  # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, 
                     activation='relu',
                     kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.5)) ##Dropout

classifier.add(Dense(output_dim=100,
                     activation='relu',
                     kernel_regularizer=regularizers.l2(0.001)))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.3))##Dropout

classifier.add(Dense(output_dim=10,
                     activation='softmax'))
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])


img_gen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True,
                             rotation_range=10, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.1, zoom_range=0.1,
                             horizontal_flip=True, vertical_flip=False, dtype=np.float32)
img_gen.fit(x_train)



earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) #earlystop

# 開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                         epochs=100, validation_data = (x_test, y_test),
                         callbacks = [earlystop])

170500096/170498071 [==============================] - 333s 2us/step
(50000, 32, 32, 3)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:49: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`


C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
500/500 [==============================] - 23s 45ms/step - loss: 2.1193 - acc: 0.3529 - val_loss: 1.6816 - val_acc: 0.4813
Epoch 2/100
 11/500 [..............................] - ETA: 5s - loss: 1.8141 - acc: 0.426

C:\Users\shinfu\anaconda3\envs\test\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `test_loss` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


500/500 [==============================] - 20s 41ms/step - loss: 1.6811 - acc: 0.4727 - val_loss: 1.4687 - val_acc: 0.5413
Epoch 3/100
500/500 [==============================] - 21s 41ms/step - loss: 1.4982 - acc: 0.5326 - val_loss: 1.3451 - val_acc: 0.5853
Epoch 4/100
500/500 [==============================] - 21s 41ms/step - loss: 1.3886 - acc: 0.5641 - val_loss: 1.2400 - val_acc: 0.6131
Epoch 5/100
500/500 [==============================] - 21s 41ms/step - loss: 1.3161 - acc: 0.5858 - val_loss: 1.1166 - val_acc: 0.6518
Epoch 6/100
500/500 [==============================] - 21s 41ms/step - loss: 1.2713 - acc: 0.5953 - val_loss: 1.2167 - val_acc: 0.6181
Epoch 7/100
500/500 [==============================] - 21s 42ms/step - loss: 1.2363 - acc: 0.6125 - val_loss: 1.1760 - val_acc: 0.6291
Epoch 8/100
500/500 [==============================] - 21s 42ms/step - loss: 1.2054 - acc: 0.6213 - val_loss: 1.0786 - val_acc: 0.6591
Epoch 9/100
500/500 [==============================] - 21s 41ms/ste